# Read / Write Operations to SAP HANA using Spark with Scala
___

### Importing Packages

In [1]:
import org.apache.spark.sql.{SparkSession,SaveMode}

### Adding SAP HANA Dependancy Jar

In [2]:
%AddJar https://repo1.maven.org/maven2/com/sap/cloud/db/jdbc/ngdbc/2.4.63/ngdbc-2.4.63.jar

Starting download from https://repo1.maven.org/maven2/com/sap/cloud/db/jdbc/ngdbc/2.4.63/ngdbc-2.4.63.jar
Finished download of ngdbc-2.4.63.jar


### Creating Spark Session

In [3]:
val spark = SparkSession.builder.appName("sap_hana").getOrCreate()

spark = org.apache.spark.sql.SparkSession@77ad9a3e


org.apache.spark.sql.SparkSession@77ad9a3e

### SAP HANA Server Credentials

In [4]:
val driver = "com.sap.db.jdbc.Driver"
var url = "jdbc:sap://192.168.2.23:39017"
var dbtable = "Lima.SLI_ATTRIBUTES"
var user = "system"
var password = "Aline-12#"
//val database = "LIMA"

driver = com.sap.db.jdbc.Driver
url = jdbc:sap://192.168.2.23:39017
dbtable = Lima.SLI_ATTRIBUTES
user = system
password = Aline-12#


Aline-12#

### Reading File

In [6]:
var filedf = spark.read.format("csv")
                   .options(Map(("header","true"),("inferSchema","true"),("delimiter","\t")))
                   .load("../Resources/SI_attr.tab")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Show

In [7]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|     Sales_Item_Id|Sales_Item_Description|  GIC|Product_Lifecycle_State|Purchasing_Group|Average Standard Production Cost|
+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                             0.0|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-112|    1500HP SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-137|     1500P SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
| IE2:15HP-TERM-112|  1500HP 50 OHM TER...|09208|               OBSOLETE|            null|                             0.0|
+-------

### Schema

In [8]:
filedf.printSchema

root
 |-- Sales_Item_Id: string (nullable = true)
 |-- Sales_Item_Description: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- Product_Lifecycle_State: string (nullable = true)
 |-- Purchasing_Group: string (nullable = true)
 |-- Average Standard Production Cost: double (nullable = true)



### Reading table from SAP HANA

In [9]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [AVERAGE_STANDARD_PRODUCTION_COST: double, GIC: string ... 4 more fields]


[AVERAGE_STANDARD_PRODUCTION_COST: double, GIC: string ... 4 more fields]

### Checking Schema of the table

In [10]:
tableDF.printSchema

root
 |-- AVERAGE_STANDARD_PRODUCTION_COST: double (nullable = true)
 |-- GIC: string (nullable = true)
 |-- SALES_ITEM_ID: string (nullable = true)
 |-- PURCHASING_GROUP: string (nullable = true)
 |-- SALES_ITEM_DESCRIPTION: string (nullable = true)
 |-- PRODUCT_LIFECYCLE_STATE: string (nullable = true)



### Changing Dataframe columns same as table columns

In [11]:
filedf = filedf.withColumnRenamed("Average Standard Production Cost","AVERAGE_STANDARD_PRODUCTION_COST")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Converting all file dataframe columns to upper case

In [12]:
filedf = filedf.toDF(filedf.columns map (_.toUpperCase): _*)

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]

### Checking Table Count before writing

In [13]:
tableDF.count()

0

### Writing to SAP HANA

In [14]:
filedf.write
  .format("jdbc")
  .options(Map(("url", url),("dbtable", dbtable),("user", user),("password", password),("driver", driver)))
  .mode(SaveMode.Append)
  .save()

### Checking Table Count after writing

In [15]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [AVERAGE_STANDARD_PRODUCTION_COST: double, GIC: string ... 4 more fields]


[AVERAGE_STANDARD_PRODUCTION_COST: double, GIC: string ... 4 more fields]

In [16]:
tableDF.count()

361560

### Closing Spark Session

In [17]:
spark.stop()